In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import clear_output
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, TimeSeriesSplit

In [2]:
df = pd.read_csv('preprocessed_train_regression.csv')
dfTest = pd.read_csv ('preprocessed_test_regression.csv')
dfToExport = pd.DataFrame (columns = ['StoreID','Month','NumberOfSales'] )

In [3]:
# Save region for each index
rows_region = df['Region']
rows_store = df['StoreID']

# Choose features
selected_features=[
    'NumberOfSales',
    'HasPromotions', 
#     'HasPromotions_yesterday',
#     'HasPromotions_tomorrow',
    'IsOpen_yesterday',
    'IsOpen_tomorrow',
    'IsHoliday_tomorrow',
    'IsHoliday_yesterday',
    'daily_sales',
    'month_avg_sales',
    'yearly_sales',
    'NearestCompetitor',
    'DayOfWeek',
    'Week',
    'Month',
    'regression_whole',
#     'regression_twomonths',
    ] # droppata dopo

df = df[selected_features]

selected_features_test=[
    'HasPromotions', 
#     'HasPromotions_yesterday',
#     'HasPromotions_tomorrow',
    'IsOpen_yesterday',
    'IsOpen_tomorrow',
    'IsHoliday_tomorrow',
    'IsHoliday_yesterday',
    'daily_sales',
    'month_avg_sales',
    'yearly_sales',
    'NearestCompetitor',
    'DayOfWeek',
    'Week',
    'Month',
    'regression_whole',
    'StoreID' # droppata dopo
#     'regression_twomonths'
    ] 
dfTest = dfTest[selected_features_test]

In [4]:
# Look for features with NaN values

null_cols = []
for col in df.columns:
    if df[col].isnull().values.any():
        null_cols.append(col)

print('Features with NaN: {}'.format(len(null_cols)))
for col in null_cols:
    print(col)
print('Num of rows containing NaNs: {}'.format(len(pd.isnull(df).any(1).nonzero()[0])))
    
# drop all rows with NaN values
# getting indices (rows) of all NaN values
inds = pd.isnull(df).any(1).nonzero()[0]
df = df.drop(df.index[inds])

Features with NaN: 1
regression_whole
Num of rows containing NaNs: 37532


In [5]:
X_train = df.drop(['NumberOfSales'], axis=1)
y_train = df["NumberOfSales"]

forest = RandomForestRegressor(n_estimators=300, max_depth=15, random_state=0, n_jobs=1, max_features='sqrt')
model = forest.fit(X_train, y_train)

X_val = dfTest.drop('StoreID', axis = 1)
y_pred = model.predict(X_val)



In [6]:
temp = pd.DataFrame (columns = ['StoreID', 'Month','NumberOfSales'] )
temp.Month = dfTest.Month
temp.StoreID = dfTest.StoreID
temp.NumberOfSales= y_pred
temp = temp.groupby(['StoreID', 'Month'], as_index=False).agg({"NumberOfSales":"sum"})
temp.NumberOfSales = temp.NumberOfSales.astype('int64')
temp

,StoreID,Month,NumberOfSales
0,1000,3,190300
1,1000,4,182852
2,1001,3,88168
3,1001,4,82639
4,1002,3,128367
5,1002,4,118502
6,1003,3,140324
7,1003,4,131595
8,1004,3,102511
9,1004,4,92175


In [7]:
temp.to_csv("testPredicted.csv", index = False)

,StoreID,Month,NumberOfSales
0,1000,3,190300
1,1000,4,182852
2,1001,3,88168
3,1001,4,82639
4,1002,3,128367
5,1002,4,118502
6,1003,3,140324
7,1003,4,131595
8,1004,3,102511
9,1004,4,92175


In [22]:
df2 = dfTest[['Month', 'StoreID', 'month_avg_sales']].drop_duplicates()

In [20]:
temp['Average Month'] = df2.reset_index().month_avg_sales

In [21]:
temp

,StoreID,Month,NumberOfSales,Average Month
0,1000,3,190300,176476.0
1,1000,4,182852,186753.0
2,1001,3,88168,55176.5
3,1001,4,82639,52291.5
4,1002,3,128367,129596.5
5,1002,4,118502,116083.5
6,1003,3,140324,134584.0
7,1003,4,131595,119086.5
8,1004,3,102511,104177.5
9,1004,4,92175,93854.0
